# L'IA et LA Sécurité des systèmes IoT
Demonstrate the practical implementation of AI for IoT Security

In [3]:
#import all libraries
import pandas as pd                                                  #python library # data manipulation and analysis[analyzing, cleaning, exploring, and manipulating data.][Tabular data with heterogeneously-typed columns, as in an SQL table or Excel spreadsheet.]
from sklearn.svm import SVC                                          #ML library  #data analysis, data preprocessing  #SVM is a supervised machine learning algorithm used for both classification and regression tasks  #SVC (Support Vector Classifier) is a specific implementation of Support Vector Machines (SVM) in Scikit-learn, used for classification tasks. 
from sklearn.model_selection import GridSearchCV                     #GridSearchCV is a tool in Scikit-learn that systematically searches through a specified set of hyperparameters for a machine learning model to determine the combination that yields the best performance.
from sklearn.metrics import classification_report, confusion_matrix   #A confusion matrix is a performance evaluation tool for classification models, summarizing the predictions made by the model and their accuracy. It provides insights into the true positives, true negatives, false positives, and false negatives.
from sklearn.model_selection import train_test_split                  #Splits a dataset into training and testing subsets.
import time
import numpy as np                                                    #Numerical python(Numpy) used for numerical computing,working with arrays.
import seaborn as sns                                                 #Seaborn is a Python data visualization library built on top of Matplotlib, designed for creating visually appealing and informative statistical graphics. 
import matplotlib.pyplot as plt                                       #data visualization library in Python    #Matplotlib is a library for creating static, interactive, and animated visualizations in Python. 
from sklearn.tree import DecisionTreeClassifier                       # A Decision Tree Classifier is a machine learning algorithm used for both classification and regression tasks. It works by splitting the dataset into subsets based on the value of input features, creating a tree-like structure of decisions. 

In [13]:
filepath = "iot23_combined.csv"  #dataset #IoT-23 is a labeled dataset with malicious and benign IoT network traffic  from Internet of Things (IoT) devices

In [14]:
df = pd.read_csv(filepath)  #read the filepath of the dataset

In [15]:
del df['Unnamed: 0']  # to delete the unnamed column (0,1,2,...column id)

In [16]:
df                              #to show the contents of columns and rows of the dataset #Pandas DataFrame, which is a 2D table-like data structure.

,ts,id.orig_h,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,1.540470e+09,192.168.1.132,0.003497,0,0,0.0,5.0,212.0,3.0,144.0,...,0,0,0,0,0,0,0,1,0,0
1,1.540470e+09,192.168.1.132,0.036724,34,311,0.0,1.0,62.0,1.0,339.0,...,0,0,0,0,0,0,0,1,0,0
2,1.540469e+09,192.168.1.132,384.518261,15072,0,0.0,48.0,16416.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
3,1.540470e+09,192.168.1.132,0.270332,48,48,0.0,1.0,76.0,1.0,76.0,...,0,0,0,0,0,0,0,1,0,0
4,1.540470e+09,192.168.1.132,0.111429,48,48,0.0,1.0,76.0,1.0,76.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446616,1.532526e+09,192.168.100.111,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1446617,1.532526e+09,192.168.100.111,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1446618,1.532526e+09,192.168.100.111,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1446619,1.532526e+09,192.168.100.111,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0


In [17]:

df['label'].value_counts()  #counting the no of occurrences of unique values in the column label , DDoS appears 138777 times in column label

label
PartOfAHorizontalPortScan     825939
Okiru                         262690
-   Benign   -                146275
DDoS                          138777
Benign                         53481
C&C                            15100
Attack                          3915
C&C-HeartBeat                    349
C&C-FileDownload                  43
C&C-Torii                         30
FileDownload                      13
C&C-HeartBeat-FileDownload         8
C&C-Mirai                          1
Name: count, dtype: int64

In [18]:
DT = DecisionTreeClassifier()  #supervised machine learning algorithm used for both classification and regression.

In [19]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = df['label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

# preparing the data and splitting it into training and testing sets

In [20]:
start = time.time()                 #Records the current time at the start of the program.
print('program start...')
print()

DT.fit(X_train, Y_train) #Training the Model to learn the patterns in the training data to make predictions.
print()

print('prediction:')    # use the trained model to Predict the labels for test data and stored in y_pred
y_pred = DT.predict(X_test)
print(y_pred)   #Prints the predicted labels for the test data.
print()

print('Score:')  #Evaluating the Model to know it's accuracy
score = DT.score(X_test,Y_test)
print(score)     #Prints the accuracy score of the model.

end = time.time()        #Records the current time at the end of the program.
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')   #Calculates and prints the total time taken for the program to run.

program start...


prediction:
['PartOfAHorizontalPortScan' 'PartOfAHorizontalPortScan'
 'PartOfAHorizontalPortScan' ... 'PartOfAHorizontalPortScan'
 'PartOfAHorizontalPortScan' '-   Benign   -']

Score:
0.7286857340361186
program end...

time cost: 
10.015539646148682 seconds


In [21]:
print("Classifiction Report :")    #classification report for evaluating the performance of a classification model.
print(classification_report(Y_test, y_pred))

Classifiction Report :


C:\Users\Afrakoma Lydia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Afrakoma Lydia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                           precision    recall  f1-score   support

           -   Benign   -       0.91      0.42      0.57     29108
                   Attack       0.99      0.99      0.99       735
                   Benign       0.97      0.89      0.93     10654
                      C&C       0.61      0.13      0.21      3125
         C&C-FileDownload       0.80      0.67      0.73        12
            C&C-HeartBeat       0.83      0.43      0.57        92
                C&C-Torii       0.00      0.00      0.00         5
                     DDoS       1.00      0.82      0.90     27685
             FileDownload       0.50      1.00      0.67         1
                    Okiru       0.85      0.00      0.00     52493
PartOfAHorizontalPortScan       0.68      1.00      0.81    165415

                 accuracy                           0.73    289325
                macro avg       0.74      0.58      0.58    289325
             weighted avg       0.78      0.73      0.65    

C:\Users\Afrakoma Lydia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
